In [1]:
pip install -q -U google-genai

In [4]:
import pandas as pd

# Carrega o arquivo CSV
df = pd.read_csv("all_msgs_whatsapp_instagram.csv")

# Mostra as colunas
print("Colunas do dataset:")
print(df.columns.tolist())

# Mostra as primeiras 5 instâncias
print("\nPrimeiras 5 instâncias:")
print(df.head())


Colunas do dataset:
['platform', 'conversation_id', 'timestamp', 'sender', 'sender_role', 'direction', 'message', 'content_type', 'has_url', 'has_media', 'is_system', 'turn_index', 'message_id', 'char_len', 'word_len']

Primeiras 5 instâncias:
    platform    conversation_id            timestamp     sender sender_role  \
0  instagram  17998336169788437  2025-05-07 09:15:00    cliente     cliente   
1  instagram  17998336169788437  2025-05-07 09:41:00  curadobia        time   
2  instagram  17998336169788437  2025-05-07 10:21:00    cliente     cliente   
3  instagram  17998336169788437  2025-05-07 10:47:00  curadobia        time   
4  instagram  17998336169788437  2025-05-07 10:57:00    cliente     cliente   

  direction                                            message content_type  \
0        in          essa gladiadora!! qual o número? tem 37??        texto   
1       out                       olá anna, tem sim tamanho 37        texto   
2        in                                 

In [8]:
mensagens_clientes = df[
    (df['sender_role'] == 'cliente') &
    (df['direction'] == 'in') &
    (df['content_type'] == 'texto')
]['message'].dropna()

print(f"🔍 TOTAL DE MENSAGENS DE CLIENTES: {len(mensagens_clientes)}")
print("=" * 80)
print()

# Mostrar todas as mensagens
for i, mensagem in enumerate(mensagens_clientes, 1):
    print(f"{i:4d}. {mensagem}")

print()
print("=" * 80)
print(f"✅ TOTAL: {len(mensagens_clientes)} mensagens de clientes encontradas")

🔍 TOTAL DE MENSAGENS DE CLIENTES: 1439

   1. essa gladiadora!! qual o número? tem 37??
   2. e tem ela preta?
   3. eu quero sim, mas vc poderia me enviar a foto do modelo só pra eu confirmar
   4. yes!!
   5. 37?? não creio
   6. tô de olho faz tempooo
   7. certo. qual o link da compra?
   8. maravilha
   9. obrigada ana!!
  10. curtiu uma mensagem
  11. compra realizada
  12. eu não recebi email confirmando a compra...
  13. 🥰
  14. olá
  15. como eu poderia rastrear minha compra?
  16. bom dia
  17. por onde posso acompanhar o envio da minha compra?
  18. obrigada
  19. boa tarde. estava previsto pra chegar hoje minha encomenda, porém nada foi entregue
  20. vc poderia checar se houve algum problema com os correios de são paulo, pois nem chegou no correio de curitiba
  21. parece que o contato por aqui não é satisfatório...
  22. bom dia
  23. vc teria alguma informação sobre a minha encomenda?
  24. 👏
  25. bom
dia 
gostei muito de uma calça mais gostaria de ter informações dos t

In [13]:
import pandas as pd
from google import genai
from pydantic import BaseModel, Field
from typing import List
import enum
from google.colab import userdata
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import time

# Configuração do cliente Gemini
client = genai.Client(api_key=userdata.get('GEMINI_API_KEY'))

# Enum com as categorias
class IntencaoCategoria(str, enum.Enum):
    DUVIDA_PRODUTO = "duvida_produto"
    SOLICITACAO_INFORMACAO = "solicitacao_informacao"
    RASTREAMENTO_PEDIDO = "rastreamento_pedido"
    PROBLEMA_TECNICO = "problema_tecnico"
    TROCA_DEVOLUCAO = "troca_devolucao"
    AGRADECIMENTO = "agradecimento"
    SAUDACAO = "saudacao"
    INTERESSE_PRODUTO = "interesse_produto"
    REPOSICAO_ESTOQUE = "reposicao_estoque"
    CONFIRMACAO = "confirmacao"
    REACAO_EMOCIONAL = "reacao_emocional"
    SOLICITACAO_CONTATO = "solicitacao_contato"
    EVENTO_PRESENCIAL = "evento_presencial"
    MENSAGEM_SISTEMA = "mensagem_sistema"
    PARCERIA_COMERCIAL = "parceria_comercial"
    NAO_IDENTIFICADO = "nao_identificado"

# Modelo para classificação
class ClassificacaoMensagem(BaseModel):
    categoria: IntencaoCategoria = Field(description="Categoria da intenção identificada")
    confianca: float = Field(description="Nível de confiança da classificação (0-1)")

# Carregar dados
df = pd.read_csv("all_msgs_whatsapp_instagram.csv")
mensagens_clientes = df[
    (df['sender_role'] == 'cliente') &
    (df['direction'] == 'in') &
    (df['content_type'] == 'texto')
]['message'].dropna().tolist()

print(f"Classificando {len(mensagens_clientes)} mensagens...")

# Função de classificação
def classificar_mensagem(mensagem: str) -> ClassificacaoMensagem:
    prompt = f"""
    Classifique a mensagem do cliente do e-commerce Curadobia na categoria mais adequada:

    MENSAGEM: "{mensagem}"

    CATEGORIAS:
    - duvida_produto: dúvidas sobre tamanhos, medidas, cores, materiais, disponibilidade
    - solicitacao_informacao: pedidos de links, preços, como comprar, onde encontrar
    - rastreamento_pedido: status de entrega, confirmação de compra, acompanhar envio
    - problema_tecnico: site não funciona, erros de pagamento, problemas técnicos
    - troca_devolucao: solicitar troca, devolução, reembolso de produtos
    - agradecimento: "obrigada", elogios, expressões de satisfação
    - saudacao: cumprimentos básicos como "olá", "bom dia", "boa tarde"
    - interesse_produto: expressar desejo/interesse em comprar algo
    - reposicao_estoque: perguntar quando produto volta, se terá reposição
    - confirmacao: "sim", "ok", "certo", confirmações simples
    - reacao_emocional: emojis, expressões emocionais curtas
    - solicitacao_contato: pedir contato por telefone/email/WhatsApp
    - evento_presencial: sobre eventos físicos, loja presencial, encontros
    - mensagem_sistema: notificações automáticas do WhatsApp/Instagram
    - parceria_comercial: propostas comerciais, spam, ofertas não solicitadas
    - nao_identificado: quando não se encaixa em nenhuma categoria acima
    """

    # Pequena pausa para evitar rate limiting
    time.sleep(0.1)

    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt,
        config={
            "response_mime_type": "application/json",
            "response_schema": ClassificacaoMensagem,
            "thinking_config": {"thinking_budget": 0}
        }
    )

    if response.parsed is None:
        return ClassificacaoMensagem(categoria=IntencaoCategoria.NAO_IDENTIFICADO, confianca=0.0)

    return response.parsed

# Função auxiliar para processar uma mensagem com índice
def processar_mensagem(dados):
    i, mensagem = dados
    try:
        resultado = classificar_mensagem(mensagem)
        return {
            'indice': i,
            'mensagem': mensagem,
            'categoria': resultado.categoria.value,
            'confianca': resultado.confianca,
            'erro': None
        }
    except Exception as e:
        return {
            'indice': i,
            'mensagem': mensagem,
            'categoria': 'nao_identificado',
            'confianca': 0.0,
            'erro': str(e)
        }

print(f"Processando {len(mensagens_clientes)} mensagens")

# Prepara dados com índices
dados_mensagens = list(enumerate(mensagens_clientes, 1))

# Paraleliza com ThreadPoolExecutor
max_workers = 8
resultados = []

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = {executor.submit(processar_mensagem, dados): dados for dados in dados_mensagens}

    for future in tqdm(as_completed(futures), total=len(futures), desc="Classificando"):
        resultado = future.result()
        resultados.append(resultado)

        if resultado['erro']:
            tqdm.write(f"❌ Erro msg {resultado['indice']}: {resultado['erro']}")

# Ordena resultados pelo índice original
resultados.sort(key=lambda x: x['indice'])

df_resultados = pd.DataFrame([
    {
        'mensagem': r['mensagem'],
        'categoria': r['categoria'],
        'confianca': r['confianca']
    }
    for r in resultados
])

# Estatísticas
erros_count = sum(1 for r in resultados if r['erro'])
print("\nRESULTADOS DA CLASSIFICAÇÃO")
print("=" * 50)
print(f"Total de mensagens: {len(df_resultados)}")
print(f"Mensagens com erro: {erros_count}")
print(f"Taxa de sucesso: {((len(df_resultados) - erros_count) / len(df_resultados) * 100):.1f}%")
print(f"Confiança média: {df_resultados['confianca'].mean():.2f}")
print()
print("DISTRIBUIÇÃO POR CATEGORIA:")
distribuicao = df_resultados['categoria'].value_counts()
for categoria, count in distribuicao.items():
    porcentagem = (count / len(df_resultados)) * 100
    print(f"{categoria:20s}: {count:4d} ({porcentagem:5.1f}%)")

# Salva resultados
df_resultados.to_csv('classificacao_taxonomia.csv', index=False)
print(f"\nResultados salvos em 'classificacao_taxonomia.csv'")

print("\n📝 EXEMPLOS POR CATEGORIA:")
print("=" * 50)
for categoria in IntencaoCategoria:
    exemplos = df_resultados[df_resultados['categoria'] == categoria.value]['mensagem'].head(3).tolist()
    if exemplos:
        print(f"\n{categoria.value}:")
        for exemplo in exemplos:
            print(f"  • {exemplo[:60]}...")

Classificando 1439 mensagens...
Processando 1439 mensagens


Classificando: 100%|██████████| 1439/1439 [01:48<00:00, 13.23it/s]


RESULTADOS DA CLASSIFICAÇÃO
Total de mensagens: 1439
Mensagens com erro: 0
Taxa de sucesso: 100.0%
Confiança média: 0.91

DISTRIBUIÇÃO POR CATEGORIA:
mensagem_sistema    :  256 ( 17.8%)
nao_identificado    :  238 ( 16.5%)
agradecimento       :  160 ( 11.1%)
reacao_emocional    :  139 (  9.7%)
saudacao            :  115 (  8.0%)
troca_devolucao     :   78 (  5.4%)
confirmacao         :   77 (  5.4%)
duvida_produto      :   77 (  5.4%)
solicitacao_informacao:   63 (  4.4%)
rastreamento_pedido :   59 (  4.1%)
reposicao_estoque   :   34 (  2.4%)
solicitacao_contato :   34 (  2.4%)
parceria_comercial  :   34 (  2.4%)
interesse_produto   :   27 (  1.9%)
problema_tecnico    :   25 (  1.7%)
evento_presencial   :   23 (  1.6%)

Resultados salvos em 'classificacao_taxonomia.csv'

📝 EXEMPLOS POR CATEGORIA:

duvida_produto:
  • essa gladiadora!! qual o número? tem 37??...
  • e tem ela preta?...
  • eu quero sim, mas vc poderia me enviar a foto do modelo só p...

solicitacao_informacao:
  • certo